# Genetic Algorithm Approach

## Setup

Import necessary libraries and load the dataset.

In [40]:
import networkx as nx
import random
import numpy as np
from itertools import permutations

In [ ]:
g = nx.read_gml("seidel_rally_graph-12.gml")
g.nodes

NodeDataView({'Weinstadt': {'coordinates': [48.229499, 16.3503737], 'district': 9.0}, 'Klub Gru': {'coordinates': [48.2259316, 16.3640482], 'district': 9.0}, 'Glashütte': {'coordinates': [48.231651, 16.3531753], 'district': 9.0}, 'Das Torberg': {'coordinates': [48.2093928, 16.3486411], 'district': 8.0}, 'Mokador 8': {'coordinates': [48.2086782, 16.3391832], 'district': 8.0}, 'u.s.w.': {'coordinates': [48.2134974, 16.3518487], 'district': 8.0}, 'Pub by Charles': {'coordinates': [48.201731, 16.3361592], 'district': 15.0}, 'Hopfen-Stuben': {'coordinates': [48.1845364, 16.3294303], 'district': 15.0}, 'Café Amadeus': {'coordinates': [48.2006881, 16.3364734], 'district': 15.0}, 'Acabar': {'coordinates': [48.2010188, 16.3468921], 'district': 7.0}, 'Ganz Wien': {'coordinates': [48.1997809, 16.3506961], 'district': 7.0}, 'Café 7*stern': {'coordinates': [48.2020085, 16.3514354], 'district': 7.0}, 'Kerstin': {'coordinates': [48.2182281, 16.3875], 'district': 2.0}, 'Prater Dome': {'coordinates': [

## Create initital Population

In [32]:
def enumerate_reversed(l):
    return zip(range(len(l)-1, -1, -1), reversed(l))

In [ ]:
def init_pop(bar_graph, n_population):
    """
    Generate a randomly initialized population of bar routes
    Input:
        1. bar_graph: Graph of all bars
        2. n_population: Size of initial population
    Output:
        List of initial population
    """

    population = []

    for _ in range(n_population):
        bar_selection = list(np.random.randint(low = 0, high = 3, size = 23))
        print(bar_selection)
        route = []

        for idx, bar in enumerate_reversed(bar_selection):
            selected_bar = [node for node, value in bar_graph.nodes(data=True) if value["district"] == idx+1][bar]
            route.append(selected_bar)

        population.append(route)

    return population

## Create Fitness Function

In [46]:
def total_distance(graph, route):
    """
    Calculate total distance for a route.
    Input:
        graph: Graph of bars
        route: List of visited bars
    Output:
        tot_dist: The total distance of the route
    """

    tot_dist = 0
    for i in range(23):
        tot_dist += graph.get_edge_data(route[i], route[i+1])["weight"]

    return tot_dist

In [ ]:
def fitness(graph, pop):
    """
    Calculate fitness for each route.
    Input: 
        graph: Graph of bars
        pop: List of routes
    Output:
        pop_fit: List of fitness
    """
    
    pop_fit = []
    for route in pop:
        tot_dist = total_distance(graph, route)
        pop.append(1 / tot_dist)

    return pop_fit

## Create Selection Process

In [48]:
def roulette_wheel(pop_fitness):
    """
    Create a roulette wheel that lets one random route survive.
    Probability is based on fitness.
    Input:
        pop_fitness: List of fitness
    Output:
        idx: Index of surviving unit
    """

    total_fitness = np.sum(pop_fitness)
    surv_prob = [i / total_fitness for i in pop_fitness]

    cumsum_surv_prob = np.cumsum(surv_prob)
    selected_mask = cumsum_surv_prob < np.random.uniform(0, 1, 1)
    idx = len(selected_mask[selected_mask == True]) - 1

    return idx

## Create Mutation Process

In [51]:
def mutation(graph, population, prob):
    """
    Mutate a random bar in a random district in each route by a certain probability.
    Input:
        graph: The graph containing the data
        population: List containing the routes
        prob: The porpability by which a mutation should happen
    Output:
        mutated_population: List containing the mutated routes
    """
    
    mutated_population = population.copy()
    mutation_mask = prob < np.random.uniform(0, 1, len(population))
    mutation_idx = np.argwhere(mutation_mask)

    for idx in mutation_idx:
        dist = np.random.randint(0, 23)
        bar = np.random.randint(0, 3)
        mutated_population[idx][dist] = graph.get_node_attribute(population[idx][dist])[bar]

    return mutated_population
        

## Create Crossover Process

In [52]:
def crossover(route_1, route_2):
    """
    Swap two routes after cutoff
    Input:
        route_1: List of bars
        route_2: Second list of bars
    Output:
        cross_1: New route
        cross_2: Second new route
    """
    
    position = np.random.randint(0, 23)
    cross_1 = route_1[:position + 1]
    cross_2 = route_2[:position + 1]
    cross_1 += route_2[position:]
    cross_2 += route_1[position:]

    return cross_1, cross_2

## Putting it all together